In [2]:
import sys
print(sys.path)

['/Users/cfruan/Documents/mlc-llm', '/Users/cfruan/miniconda3/envs/mlc-chat-venv/lib/python311.zip', '/Users/cfruan/miniconda3/envs/mlc-chat-venv/lib/python3.11', '/Users/cfruan/miniconda3/envs/mlc-chat-venv/lib/python3.11/lib-dynload', '', '/Users/cfruan/miniconda3/envs/mlc-chat-venv/lib/python3.11/site-packages']


In [3]:
sys.path.append("/Users/cfruan/Documents/tvm/python")
sys.path.append("/Users/cfruan/Documents/mlc-llm/python")

In [4]:
from mteb import MTEB
from mlc_llm.embeddings.embeddings import MLCEmbeddings
from typing import Any
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import tvm

/Users/cfruan/miniconda3/envs/mlc-chat-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### q0f32, f32 embeddings, 768 size

In [4]:
class MyModel():
    def __init__(self):
        self.model = MLCEmbeddings(
            "/Users/cfruan/Documents/mlc-llm/dist/snowflake-arctic-embed-m-q0f32-MLC",
            "/Users/cfruan/Documents/mlc-llm/dist/libs/snowflake-arctic-embed-m-q0f32-metal.so",
            device="auto",
            # debug_dir="/Users/cfruan/Documents/mlc-llm-repos/mlc-llm-head/debug",
        )
    def encode(self, sentences: list[str], **kwargs: Any):
        """Encodes the given sentences using the encoder.

        Args:
            sentences: The sentences to encode.
            prompt: The prompt to use. Useful for prompt-based models.
            **kwargs: Additional arguments to pass to the encoder.

        Returns:
            The encoded sentences.
        """
        prompt="Represent this sentence for searching relevant passages: "
        queries_with_prefix = ["{}{}".format(prompt, i) for i in sentences]
        mlc_queries = ["[CLS] " + query + " [SEP]" for query in queries_with_prefix]

        # Max batch size is 128, so we chunk it
        print(f"cur batch size: {len(mlc_queries)}")
        chunk_size = 128
        hidden_size = 768
        def ceildiv(a, b):
            return -(a // -b)
        num_chunks = ceildiv(len(mlc_queries), chunk_size)

        results = []
        for i in tqdm(range(num_chunks)):
            start = i * chunk_size
            end = min(start + chunk_size, len(mlc_queries))
            mlc_queries_chunk_i = mlc_queries[start:end]
            query_embeddings_mlc = self.model.embed(mlc_queries_chunk_i).numpy()[:,0]
            results.append(query_embeddings_mlc)

        results_np = np.vstack(results)
        assert results_np.shape == (len(sentences), hidden_size), f"Got size {query_embeddings_mlc.shape}, expected {(len(sentences), hidden_size)}"
        return results_np

model = MyModel()
evaluation = MTEB(tasks=["SciFact"])
evaluation.run(model, output_folder="results_mteb/mlc_original", eval_splits=["test"])

[2024-05-02 18:44:29] INFO chat_module.py:379: Using model folder: /Users/cfruan/Documents/mlc-llm/dist/snowflake-arctic-embed-m-q0f32-MLC
[2024-05-02 18:44:29] INFO chat_module.py:380: Using mlc chat config: /Users/cfruan/Documents/mlc-llm/dist/snowflake-arctic-embed-m-q0f32-MLC/mlc-chat-config.json
[2024-05-02 18:44:30] INFO MTEB.py:257: 

## Evaluating 1 tasks:


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- SciFact, s2p

[2024-05-02 18:44:30] INFO MTEB.py:265: 

********************** Evaluating SciFact **********************
[2024-05-02 18:44:30] INFO MTEB.py:290: Loading dataset for SciFact
[2024-05-02 18:44:30] INFO AbsTaskRetrieval.py:84: Loading Corpus...
[2024-05-02 18:44:31] INFO AbsTaskRetrieval.py:86: Loaded 5183 TEST Documents.
[2024-05-02 18:44:31] INFO AbsTaskRetrieval.py:87: Doc Example: {'id': '4983', 'title': 'Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging.', 'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm 

FileNotFoundError: Unable to find 'hf://datasets/mteb/scifact@d56462d0e63a25450459c4f213e49ffdb866f7f9/qrels/test.jsonl' with any supported extension ['.csv', '.tsv', '.json', '.jsonl', '.parquet', '.geoparquet', '.gpq', '.arrow', '.txt', '.tar', '.blp', '.bmp', '.dib', '.bufr', '.cur', '.pcx', '.dcx', '.dds', '.ps', '.eps', '.fit', '.fits', '.fli', '.flc', '.ftc', '.ftu', '.gbr', '.gif', '.grib', '.h5', '.hdf', '.png', '.apng', '.jp2', '.j2k', '.jpc', '.jpf', '.jpx', '.j2c', '.icns', '.ico', '.im', '.iim', '.tif', '.tiff', '.jfif', '.jpe', '.jpg', '.jpeg', '.mpg', '.mpeg', '.msp', '.pcd', '.pxr', '.pbm', '.pgm', '.ppm', '.pnm', '.psd', '.bw', '.rgb', '.rgba', '.sgi', '.ras', '.tga', '.icb', '.vda', '.vst', '.webp', '.wmf', '.emf', '.xbm', '.xpm', '.BLP', '.BMP', '.DIB', '.BUFR', '.CUR', '.PCX', '.DCX', '.DDS', '.PS', '.EPS', '.FIT', '.FITS', '.FLI', '.FLC', '.FTC', '.FTU', '.GBR', '.GIF', '.GRIB', '.H5', '.HDF', '.PNG', '.APNG', '.JP2', '.J2K', '.JPC', '.JPF', '.JPX', '.J2C', '.ICNS',

### q4f32_1, f32 embeddings, 768 size

In [6]:
class MyModel():
    def __init__(self):
        self.model = MLCEmbeddings(
            "/Users/cfruan/Documents/mlc-llm/dist/snowflake-arctic-embed-m-q4f32_1-MLC",
            "/Users/cfruan/Documents/mlc-llm/dist/libs/snowflake-arctic-embed-m-q4f32_1-metal.so",
            device="auto",
            # debug_dir="/Users/cfruan/Documents/mlc-llm-repos/mlc-llm-head/debug",
        )
    def encode(self, sentences: list[str], **kwargs: Any):
        """Encodes the given sentences using the encoder.

        Args:
            sentences: The sentences to encode.
            prompt: The prompt to use. Useful for prompt-based models.
            **kwargs: Additional arguments to pass to the encoder.

        Returns:
            The encoded sentences.
        """
        prompt="Represent this sentence for searching relevant passages: "
        queries_with_prefix = ["{}{}".format(prompt, i) for i in sentences]
        mlc_queries = ["[CLS] " + query + " [SEP]" for query in queries_with_prefix]

        # Max batch size is 128, so we chunk it
        print(f"cur batch size: {len(mlc_queries)}")
        chunk_size = 128
        hidden_size = 768
        def ceildiv(a, b):
            return -(a // -b)
        num_chunks = ceildiv(len(mlc_queries), chunk_size)

        results = []
        for i in tqdm(range(num_chunks)):
            start = i * chunk_size
            end = min(start + chunk_size, len(mlc_queries))
            mlc_queries_chunk_i = mlc_queries[start:end]
            query_embeddings_mlc = self.model.embed(mlc_queries_chunk_i).numpy()[:,0]
            results.append(query_embeddings_mlc)

        results_np = np.vstack(results)
        assert results_np.shape == (len(sentences), hidden_size), f"Got size {query_embeddings_mlc.shape}, expected {(len(sentences), hidden_size)}"
        return results_np

model = MyModel()
evaluation = MTEB(tasks=["SciFact"])
evaluation.run(model, output_folder="results_mteb/mlc_q4f32_1", eval_splits=["test"])

[2024-05-02 18:55:36] INFO chat_module.py:379: Using model folder: /Users/cfruan/Documents/mlc-llm/dist/snowflake-arctic-embed-m-q4f32_1-MLC
[2024-05-02 18:55:36] INFO chat_module.py:380: Using mlc chat config: /Users/cfruan/Documents/mlc-llm/dist/snowflake-arctic-embed-m-q4f32_1-MLC/mlc-chat-config.json
[2024-05-02 18:55:36] INFO MTEB.py:257: 

## Evaluating 1 tasks:


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- SciFact, s2p

[2024-05-02 18:55:36] INFO MTEB.py:265: 

********************** Evaluating SciFact **********************
[2024-05-02 18:55:36] INFO MTEB.py:290: Loading dataset for SciFact
[2024-05-02 18:55:36] INFO AbsTaskRetrieval.py:84: Loading Corpus...
[2024-05-02 18:55:38] INFO AbsTaskRetrieval.py:86: Loaded 5183 TEST Documents.
[2024-05-02 18:55:38] INFO AbsTaskRetrieval.py:87: Doc Example: {'id': '4983', 'title': 'Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging.', 'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm 

cur batch size: 300


100%|██████████| 3/3 [00:00<00:00,  3.08it/s]
[2024-05-02 18:55:42] INFO RetrievalEvaluator.py:92: Sorting Corpus by document length (Longest first)...
[2024-05-02 18:55:42] INFO RetrievalEvaluator.py:100: Encoding Corpus in batches... Warning: This might take a while!
[2024-05-02 18:55:42] INFO RetrievalEvaluator.py:101: Scoring Function: Cosine Similarity (cos_sim)
[2024-05-02 18:55:42] INFO RetrievalEvaluator.py:113: Encoding Batch 1/1...


cur batch size: 5183


100%|██████████| 41/41 [01:49<00:00,  2.67s/it]
[2024-05-02 18:57:31] INFO AbsTaskRetrieval.py:277: Time taken to retrieve: 110.63 seconds
[2024-05-02 18:57:31] INFO RetrievalEvaluator.py:433: For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
[2024-05-02 18:57:31] INFO RetrievalEvaluator.py:477: 

[2024-05-02 18:57:31] INFO RetrievalEvaluator.py:479: NDCG@1: 0.5567
[2024-05-02 18:57:31] INFO RetrievalEvaluator.py:479: NDCG@3: 0.6346
[2024-05-02 18:57:31] INFO RetrievalEvaluator.py:479: NDCG@5: 0.6632
[2024-05-02 18:57:31] INFO RetrievalEvaluator.py:479: NDCG@10: 0.6837
[2024-05-02 18:57:31] INFO RetrievalEvaluator.py:479: NDCG@20: 0.7027
[2024-05-02 18:57:31] INFO RetrievalEvaluator.py:479: NDCG@100: 0.7144
[2024-05-02 18:57:31] INFO RetrievalEvaluator.py:479: NDCG@1000: 0.7199
[2024-05-02 18:57:31] INFO RetrievalEvaluator.py:477: 

[2024-05-02 18:57:31] INFO RetrievalEvaluator.py:479: MAP@1: 0.527

{'SciFact': {'mteb_version': '1.7.58',
  'dataset_revision': '0228b52cf27578f30900b9e5271d331663a030d7',
  'mteb_dataset_name': 'SciFact',
  'test': {'ndcg_at_1': 0.55667,
   'ndcg_at_3': 0.63463,
   'ndcg_at_5': 0.66323,
   'ndcg_at_10': 0.68375,
   'ndcg_at_20': 0.70269,
   'ndcg_at_100': 0.71435,
   'ndcg_at_1000': 0.71995,
   'map_at_1': 0.52761,
   'map_at_3': 0.60502,
   'map_at_5': 0.62439,
   'map_at_10': 0.6341,
   'map_at_20': 0.64011,
   'map_at_100': 0.64196,
   'map_at_1000': 0.64218,
   'recall_at_1': 0.52761,
   'recall_at_3': 0.68767,
   'recall_at_5': 0.75706,
   'recall_at_10': 0.81761,
   'recall_at_20': 0.88967,
   'recall_at_100': 0.95,
   'recall_at_1000': 0.99333,
   'precision_at_1': 0.55667,
   'precision_at_3': 0.25,
   'precision_at_5': 0.16867,
   'precision_at_10': 0.092,
   'precision_at_20': 0.05033,
   'precision_at_100': 0.01077,
   'precision_at_1000': 0.00112,
   'mrr_at_1': 0.55667,
   'mrr_at_3': 0.62833,
   'mrr_at_5': 0.6435,
   'mrr_at_10': 0.650

### q0f32, binary embeddings, 768 size

In [5]:
class MyModel():
    def __init__(self):
        self.model = MLCEmbeddings(
            "/Users/cfruan/Documents/mlc-llm/dist/snowflake-arctic-embed-m-q0f32-MLC",
            "/Users/cfruan/Documents/mlc-llm/dist/libs/snowflake-arctic-embed-m-q0f32-metal.so",
            device="auto",
            # debug_dir="/Users/cfruan/Documents/mlc-llm-repos/mlc-llm-head/debug",
        )
    def encode(self, sentences: list[str], **kwargs: Any):
        """Encodes the given sentences using the encoder.

        Args:
            sentences: The sentences to encode.
            prompt: The prompt to use. Useful for prompt-based models.
            **kwargs: Additional arguments to pass to the encoder.

        Returns:
            The encoded sentences.
        """
        prompt="Represent this sentence for searching relevant passages: "
        queries_with_prefix = ["{}{}".format(prompt, i) for i in sentences]
        mlc_queries = ["[CLS] " + query + " [SEP]" for query in queries_with_prefix]

        # Max batch size is 128, so we chunk it
        print(f"cur batch size: {len(mlc_queries)}")
        chunk_size = 128
        hidden_size = 768
        def ceildiv(a, b):
            return -(a // -b)
        num_chunks = ceildiv(len(mlc_queries), chunk_size)

        results = []
        for i in tqdm(range(num_chunks)):
            start = i * chunk_size
            end = min(start + chunk_size, len(mlc_queries))
            mlc_queries_chunk_i = mlc_queries[start:end]
            query_embeddings_mlc = self.model.embed_binary(mlc_queries_chunk_i).numpy().astype(np.int8)

            results.append(query_embeddings_mlc)

        results_np = np.vstack(results)
        assert results_np.shape == (len(sentences), hidden_size), f"Got size {query_embeddings_mlc.shape}, expected {(len(sentences), hidden_size)}"
        return results_np

model = MyModel()
evaluation = MTEB(tasks=["SciFact"])
evaluation.run(model, output_folder="results_mteb/mlc_binary", eval_splits=["test"], score_function="hamming_distance")

[2024-05-02 18:58:41] INFO chat_module.py:379: Using model folder: /Users/cfruan/Documents/mlc-llm/dist/snowflake-arctic-embed-m-q0f32-MLC
[2024-05-02 18:58:41] INFO chat_module.py:380: Using mlc chat config: /Users/cfruan/Documents/mlc-llm/dist/snowflake-arctic-embed-m-q0f32-MLC/mlc-chat-config.json
[2024-05-02 18:58:41] INFO MTEB.py:257: 

## Evaluating 1 tasks:


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- SciFact, s2p

[2024-05-02 18:58:41] INFO MTEB.py:265: 

********************** Evaluating SciFact **********************
[2024-05-02 18:58:41] INFO MTEB.py:290: Loading dataset for SciFact
[2024-05-02 18:58:41] INFO AbsTaskRetrieval.py:84: Loading Corpus...
[2024-05-02 18:58:42] INFO AbsTaskRetrieval.py:86: Loaded 5183 TEST Documents.
[2024-05-02 18:58:42] INFO AbsTaskRetrieval.py:87: Doc Example: {'id': '4983', 'title': 'Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging.', 'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm 

cur batch size: 300


100%|██████████| 3/3 [00:00<00:00,  3.10it/s]
[2024-05-02 18:58:45] INFO RetrievalEvaluator.py:92: Sorting Corpus by document length (Longest first)...
[2024-05-02 18:58:45] INFO RetrievalEvaluator.py:100: Encoding Corpus in batches... Warning: This might take a while!
[2024-05-02 18:58:45] INFO RetrievalEvaluator.py:101: Scoring Function: Hamming distance (hamming_distance)
[2024-05-02 18:58:45] INFO RetrievalEvaluator.py:113: Encoding Batch 1/1...


cur batch size: 5183


100%|██████████| 41/41 [01:46<00:00,  2.59s/it]
[2024-05-02 19:00:32] INFO AbsTaskRetrieval.py:277: Time taken to retrieve: 107.65 seconds
[2024-05-02 19:00:32] INFO RetrievalEvaluator.py:433: For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
[2024-05-02 19:00:32] INFO RetrievalEvaluator.py:477: 

[2024-05-02 19:00:32] INFO RetrievalEvaluator.py:479: NDCG@1: 0.5000
[2024-05-02 19:00:32] INFO RetrievalEvaluator.py:479: NDCG@3: 0.5536
[2024-05-02 19:00:32] INFO RetrievalEvaluator.py:479: NDCG@5: 0.5784
[2024-05-02 19:00:32] INFO RetrievalEvaluator.py:479: NDCG@10: 0.6123
[2024-05-02 19:00:32] INFO RetrievalEvaluator.py:479: NDCG@20: 0.6266
[2024-05-02 19:00:32] INFO RetrievalEvaluator.py:479: NDCG@100: 0.6438
[2024-05-02 19:00:32] INFO RetrievalEvaluator.py:479: NDCG@1000: 0.6552
[2024-05-02 19:00:32] INFO RetrievalEvaluator.py:477: 

[2024-05-02 19:00:32] INFO RetrievalEvaluator.py:479: MAP@1: 0.475

{'SciFact': {'mteb_version': '1.7.58',
  'dataset_revision': '0228b52cf27578f30900b9e5271d331663a030d7',
  'mteb_dataset_name': 'SciFact',
  'test': {'ndcg_at_1': 0.5,
   'ndcg_at_3': 0.55363,
   'ndcg_at_5': 0.57841,
   'ndcg_at_10': 0.61231,
   'ndcg_at_20': 0.62662,
   'ndcg_at_100': 0.64377,
   'ndcg_at_1000': 0.65521,
   'map_at_1': 0.47511,
   'map_at_3': 0.53102,
   'map_at_5': 0.54721,
   'map_at_10': 0.56252,
   'map_at_20': 0.56694,
   'map_at_100': 0.56972,
   'map_at_1000': 0.57014,
   'recall_at_1': 0.47511,
   'recall_at_3': 0.59283,
   'recall_at_5': 0.65167,
   'recall_at_10': 0.75011,
   'recall_at_20': 0.80494,
   'recall_at_100': 0.89333,
   'recall_at_1000': 0.98333,
   'precision_at_1': 0.5,
   'precision_at_3': 0.21222,
   'precision_at_5': 0.14467,
   'precision_at_10': 0.08467,
   'precision_at_20': 0.04567,
   'precision_at_100': 0.0102,
   'precision_at_1000': 0.00111,
   'mrr_at_1': 0.5,
   'mrr_at_3': 0.55278,
   'mrr_at_5': 0.56644,
   'mrr_at_10': 0.58,
  

### q0f32, int8 embeddings, 768 size

In [6]:
class MyModel():
    def __init__(self):
        self.model = MLCEmbeddings(
            "/Users/cfruan/Documents/mlc-llm/dist/snowflake-arctic-embed-m-q0f32-MLC",
            "/Users/cfruan/Documents/mlc-llm/dist/libs/snowflake-arctic-embed-m-q0f32-metal.so",
            device="auto",
            # debug_dir="/Users/cfruan/Documents/mlc-llm-repos/mlc-llm-head/debug",
        )
        self.ranges = None
    def encode(self, sentences: list[str], **kwargs: Any):
        """Encodes the given sentences using the encoder.

        Args:
            sentences: The sentences to encode.
            prompt: The prompt to use. Useful for prompt-based models.
            **kwargs: Additional arguments to pass to the encoder.

        Returns:
            The encoded sentences.
        """
        prompt="Represent this sentence for searching relevant passages: "
        queries_with_prefix = ["{}{}".format(prompt, i) for i in sentences]
        mlc_queries = ["[CLS] " + query + " [SEP]" for query in queries_with_prefix]

        # Max batch size is 128, so we chunk it
        print(f"cur batch size: {len(mlc_queries)}")
        chunk_size = 128
        hidden_size = 768
        def ceildiv(a, b):
            return -(a // -b)
        num_chunks = ceildiv(len(mlc_queries), chunk_size)

        results = []
        for i in tqdm(range(num_chunks)):
            start = i * chunk_size
            end = min(start + chunk_size, len(mlc_queries))
            mlc_queries_chunk_i = mlc_queries[start:end]
            query_embeddings_mlc = self.model.embed(mlc_queries_chunk_i).numpy()[:,0]
            query_embeddings_mlc_tvm = tvm.nd.array(query_embeddings_mlc, device=self.model.device)
            if self.ranges is None:
                self.ranges = np.vstack((np.min(query_embeddings_mlc, axis=0), np.max(query_embeddings_mlc, axis=0)))
            query_int8_mlc = self.model.quantize_embeddings(query_embeddings_mlc_tvm, "int8", ranges=self.ranges).numpy()
            results.append(query_int8_mlc.astype("float32"))

        results_np = np.vstack(results)
        assert results_np.shape == (len(sentences), hidden_size), f"Got size {query_embeddings_mlc.shape}, expected {(len(sentences), hidden_size)}"
        return results_np

model = MyModel()
evaluation = MTEB(tasks=["SciFact"])
evaluation.run(model, output_folder="results_mteb/mlc_int8", eval_splits=["test"])

[2024-05-02 19:52:55] INFO chat_module.py:379: Using model folder: /Users/cfruan/Documents/mlc-llm/dist/snowflake-arctic-embed-m-q0f32-MLC
[2024-05-02 19:52:55] INFO chat_module.py:380: Using mlc chat config: /Users/cfruan/Documents/mlc-llm/dist/snowflake-arctic-embed-m-q0f32-MLC/mlc-chat-config.json
[2024-05-02 19:52:55] INFO MTEB.py:257: 

## Evaluating 1 tasks:


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- SciFact, s2p

[2024-05-02 19:52:55] INFO MTEB.py:265: 

********************** Evaluating SciFact **********************
[2024-05-02 19:52:55] INFO MTEB.py:290: Loading dataset for SciFact
[2024-05-02 19:52:55] INFO AbsTaskRetrieval.py:84: Loading Corpus...
[2024-05-02 19:52:56] INFO AbsTaskRetrieval.py:86: Loaded 5183 TEST Documents.
[2024-05-02 19:52:56] INFO AbsTaskRetrieval.py:87: Doc Example: {'id': '4983', 'title': 'Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging.', 'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm 

cur batch size: 300


100%|██████████| 3/3 [00:00<00:00,  3.06it/s]
[2024-05-02 19:52:59] INFO RetrievalEvaluator.py:92: Sorting Corpus by document length (Longest first)...
[2024-05-02 19:52:59] INFO RetrievalEvaluator.py:100: Encoding Corpus in batches... Warning: This might take a while!
[2024-05-02 19:52:59] INFO RetrievalEvaluator.py:101: Scoring Function: Cosine Similarity (cos_sim)
[2024-05-02 19:52:59] INFO RetrievalEvaluator.py:113: Encoding Batch 1/1...


cur batch size: 5183


100%|██████████| 41/41 [01:47<00:00,  2.61s/it]
[2024-05-02 19:54:46] INFO AbsTaskRetrieval.py:277: Time taken to retrieve: 108.22 seconds
[2024-05-02 19:54:46] INFO RetrievalEvaluator.py:433: For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
[2024-05-02 19:54:46] INFO RetrievalEvaluator.py:477: 

[2024-05-02 19:54:46] INFO RetrievalEvaluator.py:479: NDCG@1: 0.5900
[2024-05-02 19:54:46] INFO RetrievalEvaluator.py:479: NDCG@3: 0.6745
[2024-05-02 19:54:46] INFO RetrievalEvaluator.py:479: NDCG@5: 0.6878
[2024-05-02 19:54:46] INFO RetrievalEvaluator.py:479: NDCG@10: 0.7177
[2024-05-02 19:54:46] INFO RetrievalEvaluator.py:479: NDCG@20: 0.7314
[2024-05-02 19:54:46] INFO RetrievalEvaluator.py:479: NDCG@100: 0.7407
[2024-05-02 19:54:46] INFO RetrievalEvaluator.py:479: NDCG@1000: 0.7463
[2024-05-02 19:54:46] INFO RetrievalEvaluator.py:477: 

[2024-05-02 19:54:46] INFO RetrievalEvaluator.py:479: MAP@1: 0.560

{'SciFact': {'mteb_version': '1.7.58',
  'dataset_revision': '0228b52cf27578f30900b9e5271d331663a030d7',
  'mteb_dataset_name': 'SciFact',
  'test': {'ndcg_at_1': 0.59,
   'ndcg_at_3': 0.67449,
   'ndcg_at_5': 0.68781,
   'ndcg_at_10': 0.71765,
   'ndcg_at_20': 0.73143,
   'ndcg_at_100': 0.74069,
   'ndcg_at_1000': 0.74632,
   'map_at_1': 0.56094,
   'map_at_3': 0.64359,
   'map_at_5': 0.65399,
   'map_at_10': 0.6686,
   'map_at_20': 0.67274,
   'map_at_100': 0.67432,
   'map_at_1000': 0.67455,
   'recall_at_1': 0.56094,
   'recall_at_3': 0.73267,
   'recall_at_5': 0.76733,
   'recall_at_10': 0.85256,
   'recall_at_20': 0.90589,
   'recall_at_100': 0.95333,
   'recall_at_1000': 0.99667,
   'precision_at_1': 0.59,
   'precision_at_3': 0.26667,
   'precision_at_5': 0.17,
   'precision_at_10': 0.09633,
   'precision_at_20': 0.05117,
   'precision_at_100': 0.01083,
   'precision_at_1000': 0.00113,
   'mrr_at_1': 0.59,
   'mrr_at_3': 0.66444,
   'mrr_at_5': 0.67144,
   'mrr_at_10': 0.68268,